In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-7k1bt68h/unsloth_f5e338c4aa1647d8a5f4fc9f565adfcd
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-7k1bt68h/unsloth_f5e338c4aa1647d8a5f4fc9f565adfcd
  Resolved https://github.com/unslothai/unsloth.git to commit 05f4875aff111bf3801f6e740cc03cb7a8594c9b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 5.2 MB/s eta 0:00:00

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2


In [ ]:
import torch
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset, Dataset
import os
import gc

import os
from huggingface_hub import login

# Access the key stored in the notebook credentials
hf_token = os.environ.get("HUGGINGFACE_TOKEN")

# Use it in the login function
try:
    login(token=hf_token, add_to_git_credential=False)
    print("Hugging Face login successful.")
except Exception as e:
    print(f"Login failed: {e}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Hugging Face login successful.


In [ ]:
# --- Major Parameters ---
TARGET_LANGUAGE_CODE = "hi" # Hindi language code (use 'sw' for Swahili, 'fr' for French etc.)
TARGET_LANGUAGE_NAME = "Hindi" # For print statements

max_seq_length = 2048 # Adjust based on VRAM. Longer helps learn context.
dtype = None # Auto-detect
load_in_4bit = True # Use 4-bit quantization

# --- Use the BASE MODEL for pretraining ---
model_name = "unsloth/Qwen2-0.5B-bnb-4bit" # Use Qwen2 0.5B BASE model (4-bit)

print(f"Configuration:")
print(f"  Model Name: {model_name} (BASE model)")
print(f"  Target Language: {TARGET_LANGUAGE_NAME} ({TARGET_LANGUAGE_CODE})")
print(f"  Max Sequence Length: {max_seq_length}")
print(f"  Load in 4-bit: {load_in_4bit}")

Configuration:
  Model Name: unsloth/Qwen2-0.5B-bnb-4bit (BASE model)
  Target Language: Hindi (hi)
  Max Sequence Length: 2048
  Load in 4-bit: True


In [ ]:
import time
start_time = time.time()
print(f"Loading BASE model ({model_name}) and tokenizer...")

# Load the base model specifically
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # Optional token if login failed
)

end_time = time.time()
print(f"Base model and tokenizer loaded in {end_time - start_time:.2f} seconds.")

Loading BASE model (unsloth/Qwen2-0.5B-bnb-4bit) and tokenizer...
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Base model and tokenizer loaded in 29.43 seconds.


In [ ]:
dataset_name = "mc4"
subset_name = TARGET_LANGUAGE_CODE # Use the language code defined in Cell 3 (e.g., "hi" for Hindi)
subset_size = 20000 # Number of examples for assignment (adjust as needed)
min_length = 50     # Minimum character length filter

print(f"Loading alternative dataset: {dataset_name}, subset: {subset_name}")
print(f"Streaming dataset and taking the first {subset_size} examples...")

try:
    # Load mc4 using streaming, specifying the language code as the subset name
    streamed_dataset = load_dataset(
        dataset_name,
        subset_name, # Pass the language code here for mc4
        split="train",
        streaming=True,
        # trust_remote_code=True # May sometimes be needed for mc4 loading scripts
    )
    print("Dataset stream opened.")

    # Take a subset and filter (same logic as before)
    print(f"Filtering for examples with minimum length {min_length}...")
    # mc4 might have different column names, check features below. Assume 'text' for now.
    text_column = "text" # Default assumption for mc4
    filtered_iterable = (
        example for example in streamed_dataset.take(subset_size * 2) # Take more initially
        if len(example.get(text_column, "")) >= min_length
    )

    # Convert the filtered iterable subset to a standard Hugging Face Dataset
    print(f"Converting filtered stream to Dataset object (limit: {subset_size})...")
    dataset_list = [example for _, example in zip(range(subset_size), filtered_iterable)]

    # Create the dataset object from the list of dictionaries
    if not dataset_list:
         raise ValueError(f"No data found for language '{subset_name}' in mc4 in the first {subset_size*2} streamed examples with min length {min_length}. Try increasing subset_size or check language code.")

    dataset = Dataset.from_list(dataset_list)
    print(f"Successfully created dataset subset with {len(dataset)} examples.")

    # *** IMPORTANT: Verify the actual text column name ***
    print("\nDataset features:", dataset.features)
    if text_column not in dataset.features:
        print(f"*** WARNING: Expected text column '{text_column}' not found. Please check features and update the 'text_column' variable and potentially 'dataset_text_field' in Cell 7. ***")
        # You might need to look at dataset.features and choose the correct column containing the text.

    print(f"\nFirst example (using column '{text_column}'):")
    if text_column in dataset.features:
        print(dataset[0][text_column][:500]) # Print first 500 chars
    else:
        print("Cannot display example, text column name incorrect or missing.")


except Exception as e:
    print(f"Error loading or processing dataset '{dataset_name}' subset '{subset_name}': {e}")
    print(f"Please check the language code ('{subset_name}') and dataset availability. You might need to install 'tensorflow_datasets' if prompted by the error message for mc4.")
    # Sometimes mc4 loading depends on tfds: !pip install tensorflow_datasets
    raise

print("=== Raw Text Dataset (mc4) Loaded ===")

Loading alternative dataset: mc4, subset: hi
Streaming dataset and taking the first 20000 examples...


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

mc4.py:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

The repository for mc4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mc4.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


/root/.cache/huggingface/modules/datasets_modules/datasets/mc4/78f7a2b7e2524fa44ee464ef429d011c365f5fe129283869e7fd76856aacb83a/mc4.py:284: FutureWarning: Dataset 'mc4' is deprecated and will be deleted. Use 'allenai/c4' instead.
  warnings.warn(


Dataset stream opened.
Filtering for examples with minimum length 50...
Converting filtered stream to Dataset object (limit: 20000)...
Successfully created dataset subset with 20000 examples.

Dataset features: {'text': Value(dtype='string', id=None), 'timestamp': Value(dtype='string', id=None), 'url': Value(dtype='string', id=None)}

First example (using column 'text'):
6 साल की बच्ची अपनी मां के लिए बनी मां | UPUKLive
6 साल की बच्ची अपनी मां के लिए बनी मां
जो प्यार, करुणा और देखभाल का स्वभाव ईश्वर ने बेटियों को दिया है, वह बेटों को हासिल नहीं है। मां को ब्रेन हैमरेज हो जाने के बाद छह साल की मासूम ने जिस तरह से मां की देखभाल की, उसे देखकर लगता है कि मां असल में बेटी है और बेटी मां है। काई चेंगचेंग जब महज छह साल की थी, तो उसकी मां चेन ली को ब्रेन हैमरेज हो गया था। इसकी वजह से उनकी याददाश्त खराब हो गई।
बीते चार साल से अपनी मां को पढ़ना, लिखना और बोलना सिखाना ही क
=== Raw Text Dataset (mc4) Loaded ===


In [ ]:
# Even for continued pretraining, PEFT/LoRA is often used with Unsloth
# to make training feasible on limited hardware and manage checkpoints.
# The LoRA adapters will learn the new language patterns.
print("Configuring LoRA adapters for pretraining...")

model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Rank can be higher for pretraining (e.g., 32, 64) as we want to learn broader patterns
    lora_alpha = 64, # Adjust alpha accordingly (often 2*r)
    lora_dropout = 0, # Set to 0 for Unsloth fast patching
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
)

print("LoRA configured for pretraining:")
print(model.print_trainable_parameters())
print("=== LoRA Configuration Complete ===")

Configuring LoRA adapters for pretraining...


Unsloth 2025.3.19 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


LoRA configured for pretraining:
trainable params: 17,596,416 || all params: 511,629,184 || trainable%: 3.4393
None
=== LoRA Configuration Complete ===


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

output_directory = f"llama3_base_pretrain_{TARGET_LANGUAGE_CODE}_run1"

print(f"Configuring SFTTrainer for Continued Pretraining. Output directory: {output_directory}")

# Key difference: Use packing=True for efficient pretraining on raw text
# No custom formatting function is needed; SFTTrainer handles text packing.

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,        # The raw text dataset
    dataset_text_field = "text",    # The column containing the raw text
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,                 # <<< IMPORTANT: Enable packing for pretraining efficiency

    args = TrainingArguments(
        per_device_train_batch_size = 2,  # Keep batch size low due to sequence length
        gradient_accumulation_steps = 8,  # Increase accumulation (effective batch size 16)
        warmup_steps = 20,                # Slightly more warmup might be beneficial
        max_steps = 200,                # Set a max step count for the assignment (adjust as needed)
        num_train_epochs = 1,             # Or train for 1 epoch on the subset
        learning_rate = 1e-4,             # Learning rate can sometimes be slightly higher or lower for CPT (e.g., 5e-5 to 2e-4)
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_directory,
        save_strategy = "steps",
        save_steps = 50,                 # Save checkpoints regularly
        report_to="tensorboard",
    ),
)

print("Trainer configured for continued pretraining.")
if torch.cuda.is_available():
    gpu_stats = torch.cuda.get_device_properties(0)
    start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 3)
    print(f"Pre-Train GPU: {gpu_stats.name}. Max memory reserved: {start_gpu_memory} GB.")
print("=== Trainer Configuration Complete ===")

Configuring SFTTrainer for Continued Pretraining. Output directory: llama3_base_pretrain_hi_run1


Generating train split: 0 examples [00:00, ? examples/s]

Trainer configured for continued pretraining.
Pre-Train GPU: Tesla T4. Max memory reserved: 0.523 GB.
=== Trainer Configuration Complete ===


In [ ]:
import time

print(f"Starting continued pretraining on {TARGET_LANGUAGE_NAME} text...")
start_train_time = time.time()

# Clear cache before training
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Cleared CUDA cache.")

# Start training
trainer_stats = trainer.train()

end_train_time = time.time()
print(f"Continued Pretraining finished in {(end_train_time - start_train_time)/60:.2f} minutes.")

# Analyze memory usage
if torch.cuda.is_available():
    used_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 3)
    used_memory_for_lora = round(used_gpu_memory - start_gpu_memory, 3) if 'start_gpu_memory' in locals() else used_gpu_memory
    print(f"\nPost-Train Peak reserved memory: {used_gpu_memory} GB.")
    if 'start_gpu_memory' in locals():
         print(f"Approx. memory used for training artifacts: {used_memory_for_lora} GB.")

print("\nTraining stats:", trainer_stats)
# Expect the loss to decrease as the model learns patterns in the new language.
print("=== Continued Pretraining Complete ===")

Starting continued pretraining on Hindi text...
Cleared CUDA cache.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 25,508 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 17,596,416/5,000,000,000 (0.35% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.977300
20,1.879300
30,1.768300
40,1.732200
50,1.828900
60,1.754500
70,1.716800
80,1.776700
90,1.803700
100,1.712800


Continued Pretraining finished in 28.95 minutes.

Post-Train Peak reserved memory: 1.729 GB.
Approx. memory used for training artifacts: 1.206 GB.

Training stats: TrainOutput(global_step=200, training_loss=1.7570324039459229, metrics={'train_runtime': 1734.022, 'train_samples_per_second': 1.845, 'train_steps_per_second': 0.115, 'total_flos': 1.47650456322048e+16, 'train_loss': 1.7570324039459229, 'epoch': 0.1254508389524855})
=== Continued Pretraining Complete ===


In [ ]:
final_adapter_dir = f"{output_directory}/final_adapters"
print(f"\nSaving final LoRA adapters from pretraining to: {final_adapter_dir}")

# Save the trained LoRA adapters (capturing the learned language patterns)
model.save_pretrained(final_adapter_dir)

# Save the tokenizer as well
tokenizer.save_pretrained(final_adapter_dir)

print(f"Adapters and tokenizer saved to {final_adapter_dir}.")
print("=== Pretraining Adapters Saved ===")


Saving final LoRA adapters from pretraining to: llama3_base_pretrain_hi_run1/final_adapters
Adapters and tokenizer saved to llama3_base_pretrain_hi_run1/final_adapters.
=== Pretraining Adapters Saved ===


In [ ]:
import warnings
warnings.filterwarnings("ignore")

print("\nRunning Inference Test (Text Generation)...")

# Prepare model for inference
FastLanguageModel.for_inference(model)
model.eval()

# --- Create a prompt IN THE TARGET LANGUAGE ---
prompt_french = "Aujourd'hui, il fait très beau et le ciel"  # Translation: "Today, the weather is nice and the sky"
TARGET_LANGUAGE_NAME = "French"

print(f"Using prompt in {TARGET_LANGUAGE_NAME}: '{prompt_french}'")

# --- Tokenize the raw text prompt ---
inputs = tokenizer([prompt_french], return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

# --- Set generation parameters ---
generation_params = {
    "max_new_tokens": 50,
    "use_cache": True,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.9,
    "eos_token_id": tokenizer.eos_token_id,
    "pad_token_id": tokenizer.eos_token_id,
}

# --- Generate the response ---
print("\nGenerating continuation...")
with torch.no_grad():
    outputs = model.generate(**inputs, **generation_params)

# Decode the ENTIRE output (prompt + generation)
full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Decode only the NEWLY generated part (optional)
# generated_tokens = outputs[0][len(inputs['input_ids'][0]):]
# generated_response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("\n--- Prompt ---")
print(prompt_french)

print(f"\n--- Generated Continuation (Full Text) ---")
print(full_response)

# print(f"\n--- Generated Continuation (New Tokens Only) ---")
# print(generated_response)

# Clean up memory
del inputs, outputs
import gc
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("\n=== Pretraining Inference Test Complete ===")



Running Inference Test (Text Generation)...
Using prompt in French: 'Aujourd'hui, il fait très beau et le ciel'

Generating continuation...

--- Prompt ---
Aujourd'hui, il fait très beau et le ciel

--- Generated Continuation (Full Text) ---
Aujourd'hui, il fait très beau et le ciel est bleu, nous allons nous promener et nous regarder la mer.
J'espère que vous avez passé un bon week-end et que vous avez apprécié votre journée. Il me fait plaisir de vous dire que nous sommes en pleine

=== Pretraining Inference Test Complete ===


In [ ]:
import warnings
import torch
import gc

warnings.filterwarnings("ignore")

print("\nRunning Inference Test (Text Generation across Multiple Languages)...")

# Prepare model for inference
FastLanguageModel.for_inference(model)
model.eval()

# --- Dictionary of prompts in various languages ---
language_prompts = {
    "Hindi": "भारत एक सुंदर देश है जहाँ",
    "French": "Aujourd'hui, il fait très beau et le ciel",
    "Swahili": "Habari za asubuhi! Leo ni siku nzuri ambayo",
    "Spanish": "El sol brilla en el cielo mientras",
    "German": "Heute ist ein schöner Tag, denn"
}

# --- Loop through each language prompt ---
for lang, prompt in language_prompts.items():
    print(f"\nUsing prompt in {lang}: '{prompt}'")

    # Tokenize input
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    # Generation parameters
    generation_params = {
        "max_new_tokens": 50,
        "use_cache": True,
        "do_sample": True,
        "temperature": 0.7,
        "top_p": 0.9,
        "eos_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.eos_token_id,
    }

    # Generate output
    print("\nGenerating continuation...")
    with torch.no_grad():
        outputs = model.generate(**inputs, **generation_params)

    # Decode full response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("\n--- Prompt ---")
    print(prompt)

    print(f"\n--- Generated Continuation (Full Text) ---")
    print(full_response)

    # Clean up
    del inputs, outputs
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("\n=== Pretraining Inference Test Complete for All Languages ===")



Running Inference Test (Text Generation across Multiple Languages)...

Using prompt in Hindi: 'भारत एक सुंदर देश है जहाँ'

Generating continuation...

--- Prompt ---
भारत एक सुंदर देश है जहाँ

--- Generated Continuation (Full Text) ---
भारत एक सुंदर देश है जहाँ बहुत सुंदर रहता है. जब आपको दोस्त के राष्ट्रीय लोकसभ

Using prompt in French: 'Aujourd'hui, il fait très beau et le ciel'

Generating continuation...

--- Prompt ---
Aujourd'hui, il fait très beau et le ciel

--- Generated Continuation (Full Text) ---
Aujourd'hui, il fait très beau et le ciel est bleu. Il est super mignon ce bébé, il est très mignon. Il a un petit sourire et un grand sourire. Ce bébé est adorable et il est super mignon, il a un petit sourire et

Using prompt in Swahili: 'Habari za asubuhi! Leo ni siku nzuri ambayo'

Generating continuation...

--- Prompt ---
Habari za asubuhi! Leo ni siku nzuri ambayo

--- Generated Continuation (Full Text) ---
Habari za asubuhi! Leo ni siku nzuri ambayo kwa mmoja muda kwa kuon